In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import requests
import pickle
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 크롬 드라이버 설치경로
chrome_driver_path = 'C:/Users/sunbi/OneDrive/바탕 화면/chromedriver_win32/chromedriver.exe'



with open('2016-07-01_to_2016-12-01_nyt_twitter.p', 'rb') as f:
    data =pickle.load(f)

urls = list(data.keys())



In [3]:
def findVideolink(url) :
    try :
        chrome_driver = chrome_driver_path
        driver = webdriver.Chrome(chrome_driver) 
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.vhs-data'))) 
        src = driver.page_source
        driver.close()
        soup = BeautifulSoup(src)
        videolink = soup.select_one('p.vhs-data')
        videolink = videolink.find('a')['href']
    except :
        videolink = 'None'
    return videolink

In [4]:
def findImages(url) :
    num_of_pagedowns=50

    chrome_driver = chrome_driver_path
    driver = webdriver.Chrome(chrome_driver) 
    driver.get(url)
    body = driver.find_element_by_tag_name("body")

    while num_of_pagedowns :
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)
        num_of_pagedowns-=1

    src = driver.page_source
    driver.close()
    soup = BeautifulSoup(src)
    temp = soup.select('picture')
    imglist = list(set([i.find('img')['src'] for i in temp]))
    return imglist

In [5]:
def findThumbnail(url) :
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text)
    thumbnail = soup.find('picture').find('img')['src']
    return thumbnail

In [6]:
NYT_link =[]
videolinks=[]
uploadtime=[]
images=[]
thumbnails =[]

for url in urls :
    NYT_link.append(url)
    
    
    # 업로드 시간
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text)
    uploadtime.append(str(soup.find('time')['datetime']))
    
    # 비디오 링크 : 있다면 링크, 없다면 'None'
    videolinks.append(findVideolink(url))
    
    # 기사 사진들 링크(한 기사에 여러개) list로 저장, 썸네일 포함
    images.append(findImages(url))
    
    # 썸네일 사진 링크
    thumbnails.append(findThumbnail(url))
    
output = pd.DataFrame({'NYT link':NYT_link,'videolinks':videolinks, 'uploadtime':uploadtime, 'images':images, 'thumbnail':thumbnails})

In [ ]:
output